**Import and Setup**

In [1]:
# Import required libraries
import random
random.seed(42)

import torch
torch.manual_seed(42)

import numpy as np
np.random.seed(42)

import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Dataset path
data_dir = '/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image'
print(os.listdir(data_dir))  

# Class labels
label_dict = {'Normal': 0, 'Lung_Opacity': 1, 'Viral Pneumonia': 2} 

# Store image paths and their corresponding labels
image_paths = []
labels = []

# Loop through each class to collect images and labels
for class_name in label_dict.keys():
    class_folder = os.path.join(data_dir, class_name)

    # Check if the folder exists and list files
    if os.path.exists(class_folder):
        for filename in os.listdir(class_folder):
            if filename.endswith('.jpg'):
                image_paths.append(os.path.join(class_folder, filename))
                labels.append(label_dict[class_name])
    else:
        print(f"Folder {class_folder} does not exist.")

# Print a few image paths and labels to verify
print(image_paths[:5])  
print(labels[:5])       


['Normal', 'Lung_Opacity', 'Viral Pneumonia']
['/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image/Normal/623.jpg', '/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image/Normal/764.jpg', '/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image/Normal/1075.jpg', '/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image/Normal/771.jpg', '/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image/Normal/208.jpg']
[0, 0, 0, 0, 0]


**Preprocessing**

In [3]:
from sklearn.model_selection import train_test_split

# Dataset splitting into 70% train, 15% validation, and 15% test
# First, split into 70% train and 30% (validation + test)
train_paths, val_test_paths, train_labels, val_test_labels = train_test_split(
    image_paths, labels, test_size=0.30, random_state=42, stratify=labels
)

# Then, split the remaining 30% into 15% validation and 15% test
val_paths, test_paths, val_labels, test_labels = train_test_split(
    val_test_paths, val_test_labels, test_size=0.5, random_state=42, stratify=val_test_labels
)

# Verifying the splits
print(f"Training samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Testing samples: {len(test_paths)}")


Training samples: 2432
Validation samples: 521
Testing samples: 522


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

# Set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load pre-trained ResNet-18 model
model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze the earlier layers of the ResNet model selectively (freeze first few layers)
# Keep layers from 'layer4' (deeper layers) trainable for fine-tuning
for name, param in model.named_parameters():
    if name.startswith("layer4") or name.startswith("fc"):
        param.requires_grad = True  # Fine-tune the last layer and deeper blocks
    else:
        param.requires_grad = False  # Freeze all earlier layers

# Modify the final FC layer for the 3 classes 
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3) 

# Move model to GPU
model = model.to(device)

# Print model to confirm layer freezing
print("Layer Freezing Status:")
for name, param in model.named_parameters():
    print(f'{name}: {"Trainable" if param.requires_grad else "Frozen"}')


**Data Augmentation**

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Transforms for data augmentation and normalization
train_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for pre-trained ResNet
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Custom dataset class to load images and apply transformations
class LungDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')  # Convert to RGB as medical images are 3-channel
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Create datasets for training, validation, and testing
train_dataset = LungDiseaseDataset(train_paths, train_labels, transform=train_transforms)
val_dataset = LungDiseaseDataset(val_paths, val_labels, transform=val_test_transforms)
test_dataset = LungDiseaseDataset(test_paths, test_labels, transform=val_test_transforms)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Data loader batch outputs 
images, labels = next(iter(train_loader))
print(f"Batch shape: {images.shape}, Labels shape: {labels.shape}")


**Class Imbalance Handling**

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import numpy as np

# Compute class weights for dealing with class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)


**Model Optimization**

In [ ]:
import torch.optim as optim

# Optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=1e-4)  # Weight decay for regularization
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
current_lr = scheduler.optimizer.param_groups[0]['lr']
print(f"Current learning rate: {current_lr}")


**Model Training, Validation, and Early Stopping with TensorBoard Integration**

In [ ]:
import time
from torch.utils.tensorboard import SummaryWriter

# Function to train and validate the model
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=50, patience=12):
    writer = SummaryWriter()  # TensorBoard for visualization
    
    best_model_wts = model.state_dict()
    best_acc = 0.0
    patience_counter = 0  # For early stopping

    train_losses, val_losses, train_accs, val_accs = [], [], [], []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 20)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc.cpu())

        print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)

        val_losses.append(val_loss)
        val_accs.append(val_acc.cpu())

        print(f'Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
        writer.add_scalars('Loss', {'train': epoch_loss, 'val': val_loss}, epoch)

        # Early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()
            patience_counter = 0  
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

        # Step scheduler based on validation loss
        scheduler.step(val_loss)

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_losses, val_losses, train_accs, val_accs

# Model training
model, train_losses, val_losses, train_accs, val_accs = train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=50)


**Visualizing Training and Validation Loss and Accuracy Curves**

In [ ]:
def plot_loss_and_accuracy(train_losses, val_losses, train_accs, val_accs):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot loss
    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Validation Loss')
    ax1.set_title('Loss Curve')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Plot accuracy
    ax2.plot(train_accs, label='Train Accuracy')
    ax2.plot(val_accs, label='Validation Accuracy')
    ax2.set_title('Accuracy Curve')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    plt.show()

# Plot results after training
plot_loss_and_accuracy(train_losses, val_losses, train_accs, val_accs)


**Model Evaluation**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluate model on the test set and compute metrics
def test_model_with_metrics(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    # Collect predictions and ground truths
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # Compute precision, recall, and F1-score
    precision = precision_score(y_true, y_pred, average='weighted')  # Weighted to account for class imbalance
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"Precision (weighted): {precision:.4f}")
    print(f"Recall (weighted): {recall:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=label_dict.keys()))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_dict.keys(), yticklabels=label_dict.keys())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Evaluate model on the test set
test_model_with_metrics(model, test_loader)


In [ ]:
# Save the trained model
model_save_path = 'resnet18_lung_disease_model.pth'
torch.save(model.state_dict(), model_save_path)

# confirm the save
print(f"Model saved to {model_save_path}")
